In [1]:
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import make_blobs
from time import time
import numpy as np

In [2]:
class KMeans:
    def __init__(self, k, method = 'KMeans', replacement=True, batch_size=100, max_iter=100):
        self.k = k
        self.method = method
        self.replacement=replacement
        self.batch_size = batch_size
        self.max_iter = max_iter
        
    def fit(self, X):
        m,n = X.shape
        
        #1. randomly choose k clusters from X
        rng = np.random.RandomState(99)
        i = rng.permutation(m)[:self.k]
        self.centers = X[i]
        
        
        #having max iter makes sure it will stop eventually
        for ix in np.arange(self.max_iter):
            
            if self.method == 'KMeans':
                X_ = X
            elif self.method == 'mini_batch_KMeans':
                random = rng.randint(m)
                X_ = X[random:random+self.batch_size]
                
            

            #2. assign labels based on closest center
            labels = pairwise_distances_argmin(X_, self.centers)

            #3. find new centers
            new_centers = []
            for i in range(self.k):
                new_centers.append(X_[labels == i].mean(axis=0))

            #convert list to np.array; you can actually combine #3
            #with np.array in one sentence 
            new_centers = np.array(new_centers)

            #4 stopping criteria - if centers do not 
            #change anymore, we stop!
            #make sure to add rtol or atol since mini-batch does not converge
            if(np.allclose(self.centers, new_centers, rtol=0.2)):
                break
            else:
                self.centers = new_centers

        print(f"Done in {ix} iterations")
        
        
        #compute total within-variation score
        total_with_variation_score = 0
        labels = pairwise_distances_argmin(X, self.centers)
        #labels = pairwise_distances_argmin(X_batch, self.centers)
        
        for i in range(self.k):
            cluster_mean = X[labels==i].mean(axis=0)
            total_with_variation_score += ((X[labels==i] - cluster_mean)** 2).sum()
#             cluster_mean = X_batch[labels==i].mean(axis=0)
#             total_with_variation_score += ((X_batch[labels==i] - cluster_mean)** 2).sum()
            
        print("Total with variation score: ", total_with_variation_score)

            
            
    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers)
        
        

In [3]:
X, y_true = make_blobs(n_samples=1500, centers=4, cluster_std=0.60, random_state=0)

In [4]:
for k in range(2, 7):
    print(f"_____k = {k}_____")
    start = time()
    model = KMeans(k, method = 'mini_batch_KMeans')
    model.fit(X)
    preds = model.predict(X)
    print(f"Fit and predict time {time() - start}")

_____k = 2_____
Done in 6 iterations
Total with variation score:  5859.027548790498
Fit and predict time 0.014862060546875
_____k = 3_____
Done in 3 iterations
Total with variation score:  2849.7266714358066
Fit and predict time 0.006865024566650391
_____k = 4_____
Done in 9 iterations
Total with variation score:  1007.7374341654453
Fit and predict time 0.009994983673095703
_____k = 5_____
Done in 9 iterations
Total with variation score:  920.8127407429872
Fit and predict time 0.009394168853759766
_____k = 6_____
Done in 3 iterations
Total with variation score:  841.4034939553978
Fit and predict time 0.009680032730102539


With Mini_Batch_KMeans, k = 3 is the best because less iteration and no longer fit time.


In [5]:
for k in range(2, 7):
    print(f"_____k = {k}_____")
    start = time()
    model = KMeans(k, method = 'KMeans')
    model.fit(X)
    preds = model.predict(X)
    print(f"Fit and predict time {time() - start}")

_____k = 2_____
Done in 2 iterations
Total with variation score:  7257.279283725391
Fit and predict time 0.008738040924072266
_____k = 3_____
Done in 7 iterations
Total with variation score:  2494.026049977925
Fit and predict time 0.013726949691772461
_____k = 4_____
Done in 6 iterations
Total with variation score:  1006.5440417040327
Fit and predict time 0.010484933853149414
_____k = 5_____
Done in 3 iterations
Total with variation score:  919.6988320968405
Fit and predict time 0.0073740482330322266
_____k = 6_____
Done in 3 iterations
Total with variation score:  855.6152494347657
Fit and predict time 0.0074481964111328125


With KMeans, k = 2 is the best because of less iteration.